# 개요

## 코스닥

In [55]:
import pandas as pd
import yfinance as yf

start = '2024-11-12'
end = '2024-11-23'

tickers = pd.read_csv(f'/Users/leesangwon/Downloads/코스닥이상치.csv', encoding="utf-8")
# 티커 칼럼 리스트화
tickers_list = tickers['Original_Ticker'].tolist() if 'Original_Ticker' in tickers.columns else tickers.iloc[:, 0].tolist()
symbol = tickers_list

stock = yf.download(symbol, start, end)
stock_kosdaq = stock['Adj Close']

# 날짜 인덱스를 칼럼으로 변환
stock_kosdaq_reset = stock_kosdaq.reset_index()

stock_kosdaq_reset

# CSV 파일로 저장
stock_kosdaq_reset.to_csv('/Users/leesangwon/Downloads/코스닥실제주가_1112_1119.csv', index=False)

[*********************100%***********************]  49 of 49 completed


## 코스피

In [56]:
import pandas as pd
import yfinance as yf

start = '2024-11-12'
end = '2024-11-20'

tickers = pd.read_csv(f'/Users/leesangwon/Downloads/코스피이상치.csv', encoding="cp949")
# 티커 칼럼 리스트화
tickers_list = tickers['Original Ticker'].tolist() if 'Original Ticker' in tickers.columns else tickers.iloc[:, 0].tolist()
symbol = tickers_list

stock = yf.download(symbol, start, end)
stock_kospi= stock['Adj Close']

# 날짜 인덱스를 칼럼으로 변환
stock_kospi_reset = stock_kospi.reset_index()

stock_kospi

# CSV 파일로 저장
stock_kospi_reset.to_csv('/Users/leesangwon/Downloads/코스피실제주가_1112_1119.csv', index=False)


[*********************100%***********************]  17 of 17 completed


## 예측주가

In [61]:
df = pd.read_csv(f'/Users/leesangwon/Downloads/코스피이상치.csv', encoding="cp949")


In [62]:
# Original_Ticker 칼럼의 고유값을 리스트로 변환
original_tickers = df['Original Ticker'].tolist()

# 결과 출력
original_tickers

['016450.KS',
 '004870.KS',
 '095720.KS',
 '006490.KS',
 '068290.KS',
 '008730.KS',
 '170900.KS',
 '003160.KS',
 '016880.KS',
 '010690.KS',
 '378850.KS',
 '007280.KS',
 '000520.KS',
 '020120.KS',
 '002880.KS',
 '006740.KS',
 '028100.KS']

In [63]:
# 필요한 라이브러리 로드
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

# 티커 리스트
symbols = original_tickers


# 예측 결과를 담을 DataFrame
forecast_results = pd.DataFrame()

# 티커별로 예측 수행
for symbol in symbols:
    # 1. 데이터 다운로드 (수정 종가 사용)
    data = yf.download(symbol, start="2024-04-10", end="2024-10-11")
    data = data['Adj Close']
    
    if data.empty:
        print(f"데이터 다운로드 실패: {symbol}")
        continue
    else:
        print(f"{symbol} 데이터: {data.head()}")
    
    # 2. 데이터 전처리
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data.values.reshape(-1, 1))

    # 3. 학습 데이터 생성
    sequence_length = 60
    X_train, y_train = [], []
    for i in range(sequence_length, len(scaled_data)):
        X_train.append(scaled_data[i-sequence_length:i, 0])
        y_train.append(scaled_data[i, 0])

    X_train, y_train = np.array(X_train), np.array(y_train)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

    # 4. LSTM 모델 생성
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dense(units=25))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')

    # 5. 모델 학습
    model.fit(X_train, y_train, batch_size=32, epochs=20)

    # 6. 예측 준비
    test_data = scaled_data[-sequence_length:]
    test_data = np.array([test_data[:, 0]])
    test_data = np.reshape(test_data, (test_data.shape[0], test_data.shape[1], 1))

    # 7. 29일 예측
    predictions = []
    for _ in range(29):
        next_pred = model.predict(test_data)[0][0]
        predictions.append(next_pred)
        test_data = np.append(test_data[:, 1:, :], [[[next_pred]]], axis=1)

    # 8. 예측 결과 복원
    predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

    # 9. 예측 결과 시각화
    forecast_index = pd.date_range(start=data.index[-1], periods=30, freq='B')[1:]
    
    # 예측 결과를 DataFrame에 저장
    forecast_df = pd.DataFrame({'Date': forecast_index, f'{symbol} Forecasted Price': predictions.flatten()})
    forecast_df.set_index('Date', inplace=True)

    # 결과를 종합 DataFrame에 추가
    if forecast_results.empty:
        forecast_results = forecast_df
    else:
        forecast_results = pd.merge(forecast_results, forecast_df, left_index=True, right_index=True, how='outer')

# 예측 결과 출력
forecast_results

[*********************100%***********************]  1 of 1 completed

016450.KS 데이터: Date
2024-04-11    4675.0
2024-04-12    4585.0
2024-04-15    4575.0
2024-04-16    4525.0
2024-04-17    4645.0
Name: Adj Close, dtype: float64
Epoch 1/20



/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.3394 
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.1483
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0452
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0343
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0629
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0293
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0198
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0216
Epoch 9/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0295
Epoch 10/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0305
Epoch 11/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0324
Epoch 12/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0256
Epoch 13/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0192
Epoch 14/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0181
Epoch 15/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0185
Epoch 16/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms

[*********************100%***********************]  1 of 1 completed

...

[*********************100%***********************]  1 of 1 completed

028100.KS 데이터: Date
2024-04-11    12570.0
2024-04-12    12680.0
2024-04-15    12500.0
2024-04-16    12060.0
2024-04-17    12160.0
Name: Adj Close, dtype: float64
Epoch 1/20



/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.3503 
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.1780
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0581
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0686
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0828
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0581
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0477
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0543
Epoch 9/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0609
Epoch 10/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0635
Epoch 11/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0549
Epoch 12/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0444
Epoch 13/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0441
Epoch 14/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0480
Epoch 15/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0516
Epoch 16/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms

,016450.KS Forecasted Price,004870.KS Forecasted Price,095720.KS Forecasted Price,006490.KS Forecasted Price,068290.KS Forecasted Price,008730.KS Forecasted Price,170900.KS Forecasted Price,003160.KS Forecasted Price,016880.KS Forecasted Price,010690.KS Forecasted Price,378850.KS Forecasted Price,007280.KS Forecasted Price,000520.KS Forecasted Price,020120.KS Forecasted Price,002880.KS Forecasted Price,006740.KS Forecasted Price,028100.KS Forecasted Price
Date,,,,,,,,,,,,,,,,,
2024-10-11,4506.418457,542.955505,1736.502441,1566.646851,14852.553711,22513.753906,71763.093750,14489.668945,928.623291,8836.089844,4050.024658,1512.493774,17558.943359,3550.747803,1030.781372,1382.767212,12507.388672
2024-10-14,4505.831055,545.955017,1735.372314,1634.316772,14849.387695,22526.144531,71673.476562,14539.636719,927.306641,8852.316406,4062.500488,1510.616577,17829.082031,3552.749512,1031.135742,1379.869995,12494.659180
2024-10-15,4505.159668,548.127075,1734.355347,1697.850098,14845.684570,22524.398438,71610.484375,14566.657227,925.770874,8858.357422,4071.927246,1509.335205,18129.986328,3553.547607,1030.949707,1378.384033,12489.718750
2024-10-16,4504.451660,549.710144,1733.432861,1759.473267,14841.742188,22513.087891,71566.226562,14577.814453,924.104370,8857.637695,4079.084473,1508.539673,18452.898438,3553.459961,1030.379883,1377.733521,12490.147461
2024-10-17,4503.735840,550.874756,1732.589478,1820.839600,14837.713867,22495.435547,71534.953125,14577.925781,922.362427,8852.469727,4084.534668,1508.117920,18792.365234,3552.730957,1029.542114,1377.562866,12494.212891
2024-10-18,4503.031250,551.743286,1731.813232,1883.178101,14833.682617,22473.693359,71512.593750,14570.345703,920.579407,8844.436523,4088.683350,1507.970093,19144.435547,3551.546387,1028.521118,1377.651855,12500.675781
2024-10-21,4502.347656,552.403137,1731.094727,1947.421265,14829.697266,22449.402344,71496.320312,14557.480469,918.777344,8834.622070,4091.836182,1508.014771,19506.224609,3550.046631,1027.378906,1377.865723,12508.641602
2024-10-22,4501.690430,552.916016,1730.426636,2014.282104,14825.778320,22423.611328,71484.195312,14541.070312,916.970215,8823.776367,4094.211914,1508.188599,19875.576172,3548.337402,1026.160156,1378.123901,12517.479492
2024-10-23,4501.063477,553.325623,1729.803101,2084.297607,14821.941406,22397.064453,71474.890625,14522.409180,915.167175,8812.411133,4095.967285,1508.444336,20250.791016,3546.497314,1024.896973,1378.380127,12526.744141


In [66]:
# 날짜 인덱스를 칼럼으로 변환
forecast_results_reset = forecast_results.reset_index()

forecast_results_reset

forecast_results_reset.to_csv('/Users/leesangwon/Downloads/코스피예측주가_1011_1120.csv', index=False)
